In [20]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import copy

In [21]:
df = pd.read_csv('./OnlineNewsPopularity.csv')
df = df.drop(df.columns[list(range(0,2))+list(range(4,7))+list(range(13,39))],axis = 1)
dataset = np.array(df)

## Data Preprocessing

In [22]:
def augment_by_one(data):
    ones = np.ones(data.shape[0]).reshape(-1,1)
    augmented_data = np.hstack((ones,data))
    return augmented_data
def split_attribute_and_label(data):
    x = data[:,:-1]
    y = data[:,-1:]
    return x,y

In [23]:
# shuffle the dataset
np.random.seed(42)
np.random.shuffle(dataset)
train = dataset[:31715]
dev = dataset[31715:31715+3964]
test = dataset[31715+3964:31715+3964+3965]
f'the train size is {train.shape[0]}, the dev shape is {dev.shape[0]}, and the test shape is {test.shape[0]}'

'the train size is 31715, the dev shape is 3964, and the test shape is 3965'

In [24]:
stdScaler = StandardScaler()
stdScaler.fit(train)
centered_train = stdScaler.transform(train)
centered_dev = stdScaler.transform(dev)
centered_test = stdScaler.transform(test)
augmented_train = augment_by_one(centered_train)
augmented_dev = augment_by_one(centered_dev)
augmented_test = augment_by_one(centered_test)

In [25]:
train_x ,train_y = split_attribute_and_label(augmented_train)
dev_x ,dev_y = split_attribute_and_label(augmented_dev)
test_x ,test_y = split_attribute_and_label(augmented_test)

In [26]:
train_x

array([[ 1.        ,  0.76066508,  1.00690312, ..., -0.26330178,
         0.83711369, -0.69029101],
       [ 1.        , -0.65736617, -0.08776171, ..., -0.26330178,
         0.83711369, -0.69029101],
       [ 1.        , -0.65736617, -0.66716439, ..., -0.26330178,
         0.83711369, -0.69029101],
       ...,
       [ 1.        , -0.65736617,  0.90855432, ..., -0.26330178,
         0.83711369, -0.69029101],
       [ 1.        ,  0.28798799,  3.06153769, ..., -1.76983497,
        -0.75038339,  1.07998198],
       [ 1.        ,  1.23334216, -0.60516189, ...,  0.39580649,
        -0.88267481,  0.08420343]])

## Part I: Linear Regression via QR Factorization

In [27]:
def proj(X_i,U_j):
    return np.dot(X_i.T,U_j)/(np.dot(U_j.T,U_j))
def norm(X):
    return np.sqrt(np.dot(X.T,X))


In [28]:
Q = np.zeros(shape = train_x.shape)
for i in range(train_x.shape[1]):
    X_i = np.array(train_x[:,i],copy = True,dtype=np.float64)
    X_i_proj = np.array(train_x[:,i],copy = True,dtype=np.float64)
    j = 0
    while j < i:
        U_j = np.array(Q[:,j],copy = True)
        X_i -= proj(X_i_proj,U_j)*U_j
        j += 1
    Q[:,i] = X_i

In [29]:
R = np.eye(train_x.shape[1],dtype = np.float64)
for i in range(train_x.shape[1]):
    for j in range(i+1,train_x.shape[1]):
        R[i,j] = proj(train_x[:,j],Q[:,i]).astype(np.float64)

# Check the validity of QR decomposition by substracting the product with the training set

In [30]:
np.sum(np.dot(Q,R)-train_x)

4.0265685920346395e-13

In [31]:
delta_inv = np.eye(train.shape[1])
for i in range(train_x.shape[1]):
    delta_inv[i][i] = 1/(norm(Q[:,i])**2)

In [32]:
w = np.dot(np.dot(np.dot(np.linalg.inv(R),delta_inv),Q.T),train_y)

# Print the weight martix

In [33]:
w

array([[ 3.11117068e-07],
       [ 9.66295060e-03],
       [ 3.64901926e-03],
       [ 3.79142414e-02],
       [-2.14405270e-02],
       [ 1.35098596e-02],
       [ 5.81755383e-03],
       [-2.12810592e-02],
       [ 1.76019940e-02],
       [ 3.65084821e+09],
       [ 3.07339877e+09],
       [ 3.91819926e+09],
       [ 4.09637353e+09],
       [ 4.01558005e+09],
       [ 4.49606771e-02],
       [-2.69729883e-03],
       [-1.30770837e-02],
       [ 3.02448432e-03],
       [-2.05323998e-02],
       [-1.48416003e-02],
       [ 1.29739068e-04],
       [-7.77602841e-03],
       [-3.71740947e-03],
       [-1.69262828e-02],
       [ 5.61358945e-03],
       [-8.84668491e-03],
       [-1.93611374e-03],
       [ 3.74773866e-03],
       [ 5.05737254e-03],
       [ 1.30838954e-02]])

In [34]:
def SSE(pred,y):
    return np.sum(np.square(pred-y))
def TSS(pred,y):
    return np.sum(np.square(pred-y.mean()))

In [35]:
pred_test = np.dot(test_x,w)

In [36]:
print(f'SSE is {SSE(pred_test,test_y)}')
print(f'TSS is {TSS(pred_test,test_y)}')

SSE is 1.92997906307328e+20
TSS is 1.9299790630133747e+20


In [37]:
R2 = (TSS(pred_test,test_y)-SSE(pred_test,test_y))/TSS(pred_test,test_y)
print(f'R square is {R2}')

R square is -3.1039272926861205e-11


## Part II: Linear Regression with Regularization

In [38]:
sse_list = []
for alpha in [0,10,100,1000,10000,100000]:
    t = 0
    eta = 1e-6
    epsilon = 1e-5
    w_prev = np.zeros((train_x.shape[1],1))
    w_next = np.random.rand(train_x.shape[1],1)
    t = 0
    while norm(w_next-w_prev) > epsilon:
        gradient_w = - np.dot(train_x.T, train_y) + np.dot(np.dot(train_x.T, train_x), w_next) + (alpha * w_next)
        w_prev = copy.deepcopy(w_next)
        w_next = w_next - eta * gradient_w
        t+=1
    dev_pred = np.dot(dev_x, w_next)
    dev_sse = SSE(dev_pred,dev_y)
    dev_tss = TSS(dev_pred,dev_y)
    dev_r2 = (dev_tss-dev_sse)/dev_tss
    sse_list.append(dev_sse)
    print(f'stop at epoch {t}') 
    print(f'the SSE is {dev_sse} when alpha is {alpha}')
    print(f'the R2 is {dev_r2} when alpha is {alpha}')

stop at epoch 4108
the SSE is 4145.757139949182 when alpha is 0
the R2 is -87.07597151071629 when alpha is 0
stop at epoch 4549
the SSE is 4146.157692073064 when alpha is 10
the R2 is -86.95040276608322 when alpha is 10
stop at epoch 20961
the SSE is 4145.92249950945 when alpha is 100
the R2 is -87.43097506176518 when alpha is 100
stop at epoch 4808
the SSE is 4145.330511068926 when alpha is 1000
the R2 is -91.3528240306676 when alpha is 1000
stop at epoch 663
the SSE is 4144.361688342447 when alpha is 10000
the R2 is -128.9969286547561 when alpha is 10000
stop at epoch 90
the SSE is 4157.51466037324 when alpha is 100000
the R2 is -672.4812478585645 when alpha is 100000


# We find a local minimum around alpha = 10000. we then search from alpha = 7000 to alpha = 13000

In [39]:
sse_list = []
for alpha in range(7000,13000,100):
    t = 0
    eta = 1e-6
    epsilon = 1e-5
    w_prev = np.zeros((train_x.shape[1],1))
    w_next = np.random.rand(train_x.shape[1],1)
    t = 0
    while norm(w_next-w_prev) > epsilon:
        gradient_w = - np.dot(train_x.T, train_y) + np.dot(np.dot(train_x.T, train_x), w_next) + (alpha * w_next)
        w_prev = copy.deepcopy(w_next)
        w_next = w_next - eta * gradient_w
        t+=1
    dev_pred = np.dot(dev_x, w_next)
    dev_sse = SSE(dev_pred,dev_y)
    dev_tss = TSS(dev_pred,dev_y)
    dev_r2 = (dev_tss-dev_sse)/dev_tss
    sse_list.append(dev_sse)
    print(f'stop at epoch {t}') 
    print(f'the SSE is {dev_sse} when alpha is {alpha}')
    print(f'the R2 is {dev_r2} when alpha is {alpha}')

stop at epoch 930
the SSE is 4144.222474035609 when alpha is 7000
the R2 is -116.40191168477372 when alpha is 7000
stop at epoch 967
the SSE is 4144.229650517257 when alpha is 7100
the R2 is -116.81393223215515 when alpha is 7100
stop at epoch 886
the SSE is 4144.236136515439 when alpha is 7200
the R2 is -117.2297853747514 when alpha is 7200
stop at epoch 936
the SSE is 4144.227934967309 when alpha is 7300
the R2 is -117.65119609370282 when alpha is 7300
stop at epoch 905
the SSE is 4144.2255643410235 when alpha is 7400
the R2 is -118.06961166565745 when alpha is 7400
stop at epoch 896
the SSE is 4144.231129418384 when alpha is 7500
the R2 is -118.48505512296151 when alpha is 7500
stop at epoch 863
the SSE is 4144.240423460927 when alpha is 7600
the R2 is -118.89949721512923 when alpha is 7600
stop at epoch 858
the SSE is 4144.239280131044 when alpha is 7700
the R2 is -119.32243793065044 when alpha is 7700
stop at epoch 833
the SSE is 4144.235173176723 when alpha is 7800
the R2 is -119

In [40]:
np.argmin(sse_list)

0

# This result suggests best alpha Probably occur before 7000, so we try the values from 3000 to 7000

In [41]:
sse_list = []
for alpha in range(3000,7001,100):
    t = 0
    eta = 1e-6
    epsilon = 1e-5
    w_prev = np.zeros((train_x.shape[1],1))
    w_next = np.random.rand(train_x.shape[1],1)
    t = 0
    while norm(w_next-w_prev) > epsilon:
        gradient_w = - np.dot(train_x.T, train_y) + np.dot(np.dot(train_x.T, train_x), w_next) + (alpha * w_next)
        w_prev = copy.deepcopy(w_next)
        w_next = w_next - eta * gradient_w
        t+=1
    dev_pred = np.dot(dev_x, w_next)
    dev_sse = SSE(dev_pred,dev_y)
    dev_tss = TSS(dev_pred,dev_y)
    dev_r2 = (dev_tss-dev_sse)/dev_tss
    sse_list.append(dev_sse)
    print(f'stop at epoch {t}') 
    print(f'the SSE is {dev_sse} when alpha is {alpha}')
    print(f'the R2 is {dev_r2} when alpha is {alpha}')

stop at epoch 1774
the SSE is 4144.612572004861 when alpha is 3000
the R2 is -99.78133368373061 when alpha is 3000
stop at epoch 1928
the SSE is 4144.597626303299 when alpha is 3100
the R2 is -100.19476212686631 when alpha is 3100
stop at epoch 1900
the SSE is 4144.56922014117 when alpha is 3200
the R2 is -100.61432410284802 when alpha is 3200
stop at epoch 1711
the SSE is 4144.539299952674 when alpha is 3300
the R2 is -101.0348497654293 when alpha is 3300
stop at epoch 1785
the SSE is 4144.527937456993 when alpha is 3400
the R2 is -101.446652058725 when alpha is 3400
stop at epoch 1635
the SSE is 4144.510238998453 when alpha is 3500
the R2 is -101.86197228898347 when alpha is 3500
stop at epoch 1740
the SSE is 4144.493529848428 when alpha is 3600
the R2 is -102.27712379990413 when alpha is 3600
stop at epoch 1563
the SSE is 4144.457922186108 when alpha is 3700
the R2 is -102.69908242082506 when alpha is 3700
stop at epoch 1580
the SSE is 4144.457242468116 when alpha is 3800
the R2 is 

In [42]:
np.argmin(sse_list)

38

# The best alpha is 6800

In [45]:
alpha = 6700
t = 0
eta = 1e-6
epsilon = 1e-5
w_prev = np.zeros((train_x.shape[1],1))
w_next = np.random.rand(train_x.shape[1],1)
t = 0
while norm(w_next-w_prev) > epsilon:
    gradient_w = - np.dot(train_x.T, train_y) + np.dot(np.dot(train_x.T, train_x), w_next) + (alpha * w_next)
    w_prev = copy.deepcopy(w_next)
    w_next = w_next - eta * gradient_w
    t+=1
test_pred = np.dot(test_x, w_next)
test_sse = SSE(test_pred,test_y)
test_tss = TSS(test_pred,test_y)
test_r2 = (test_tss-test_sse)/test_tss
sse_list.append(test_sse)
print(f'stop at epoch {t}') 
print(f'the SSE is {test_sse} on test set when alpha is {alpha}')
print(f'the R2 is {test_r2} on test set when alpha is {alpha}')

stop at epoch 965
the SSE is 1910.7299413873384 on test set when alpha is 6700
the R2 is -51.10393011449761 on test set when alpha is 6700


In [46]:
print('The corresponding matrix is')
print(w_next)

The corresponding matrix is
[[ 4.81085117e-18]
 [ 7.11941138e-03]
 [ 1.08348570e-03]
 [ 3.00498286e-02]
 [-1.51070902e-02]
 [ 1.39280460e-02]
 [ 6.70577385e-03]
 [-1.89487970e-02]
 [ 1.53039195e-02]
 [ 5.71954949e-03]
 [-1.15829617e-02]
 [-3.29373336e-02]
 [ 4.00424207e-02]
 [-1.98655442e-03]
 [ 2.90082861e-02]
 [-2.41320609e-04]
 [-8.06506250e-03]
 [ 5.04893677e-04]
 [-1.20180102e-02]
 [-5.62797614e-03]
 [ 8.49181162e-04]
 [-5.11850332e-03]
 [-2.51493527e-03]
 [-1.29520895e-02]
 [-1.26672962e-03]
 [-8.71200738e-03]
 [ 2.16892913e-03]
 [ 3.42211048e-03]
 [ 4.50369370e-03]
 [ 1.00597472e-02]]
